# Crop the Image and Split into Tubes

## Import everything

In [1]:
%reset

import cv2 as cv
#from dt_apriltags import Detector
from pupil_apriltags import Detector
import numpy as np
import matplotlib.pyplot as plt
import os
from numpy.ma.core import size
from torch._C import NoneType
import PIL.Image
import utility

from carvekit.api.interface import Interface
from carvekit.ml.wrap.fba_matting import FBAMatting
from carvekit.ml.wrap.tracer_b7 import TracerUniversalB7
from carvekit.pipelines.postprocessing import MattingMethod
from carvekit.pipelines.preprocessing import PreprocessingStub
from carvekit.trimap.generator import TrimapGenerator


#@title Download all models
from carvekit.ml.files.models_loc import tracer_b7_pretrained

tracer_b7_pretrained()

WindowsPath('C:/Users/Ke Xu/.cache/carvekit/checkpoints/tracer_b7/tracer_b7.pth')

## Import Raw Images and Crop

In [2]:
path = 'Images'
img_nameList = utility.find_img_name(f'{path}/Raw')
len(img_nameList)

49

In [3]:
img_nameList

['PPPPPYYY (1)',
 'PPPPPYYY (10)',
 'PPPPPYYY (11)',
 'PPPPPYYY (12)',
 'PPPPPYYY (13)',
 'PPPPPYYY (14)',
 'PPPPPYYY (15)',
 'PPPPPYYY (16)',
 'PPPPPYYY (17)',
 'PPPPPYYY (18)',
 'PPPPPYYY (19)',
 'PPPPPYYY (2)',
 'PPPPPYYY (20)',
 'PPPPPYYY (21)',
 'PPPPPYYY (22)',
 'PPPPPYYY (23)',
 'PPPPPYYY (24)',
 'PPPPPYYY (25)',
 'PPPPPYYY (26)',
 'PPPPPYYY (27)',
 'PPPPPYYY (28)',
 'PPPPPYYY (29)',
 'PPPPPYYY (3)',
 'PPPPPYYY (30)',
 'PPPPPYYY (4)',
 'PPPPPYYY (5)',
 'PPPPPYYY (6)',
 'PPPPPYYY (7)',
 'PPPPPYYY (8)',
 'PPPPPYYY (9)',
 'PYPYPPYY (1)',
 'PYPYPPYY (10)',
 'PYPYPPYY (11)',
 'PYPYPPYY (12)',
 'PYPYPPYY (13)',
 'PYPYPPYY (14)',
 'PYPYPPYY (15)',
 'PYPYPPYY (16)',
 'PYPYPPYY (17)',
 'PYPYPPYY (18)',
 'PYPYPPYY (19)',
 'PYPYPPYY (2)',
 'PYPYPPYY (3)',
 'PYPYPPYY (4)',
 'PYPYPPYY (5)',
 'PYPYPPYY (6)',
 'PYPYPPYY (7)',
 'PYPYPPYY (8)',
 'PYPYPPYY (9)']

# Edge Enhancement

In [5]:
for img_name in img_nameList:

  print(f'\nEnhancing {img_name}\n')

  for tube_num in range(8):
    image = cv.imread(f'{path}\Split\{img_name}_Tube{tube_num+1}.jpg')
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)[:,:,1]

    blurred_image = cv.GaussianBlur(gray_image, (9, 9), 0)
    edges = cv.Canny(blurred_image, threshold1=15, threshold2=30)

    edge_color = [0, 0, 0]  # BGR format
    enhanced = image.copy()
    enhanced[edges > 0] = edge_color

    cv.imwrite(f'{path}\Split_enhanced\{img_name}_Tube{tube_num+1}_enhanced.jpg', enhanced)


Enhancing PPPPPYYY (1)


Enhancing PPPPPYYY (10)


Enhancing PPPPPYYY (11)


Enhancing PPPPPYYY (12)


Enhancing PPPPPYYY (13)


Enhancing PPPPPYYY (14)


Enhancing PPPPPYYY (15)


Enhancing PPPPPYYY (16)


Enhancing PPPPPYYY (17)


Enhancing PPPPPYYY (18)


Enhancing PPPPPYYY (19)


Enhancing PPPPPYYY (2)


Enhancing PPPPPYYY (20)


Enhancing PPPPPYYY (21)


Enhancing PPPPPYYY (22)


Enhancing PPPPPYYY (23)


Enhancing PPPPPYYY (24)


Enhancing PPPPPYYY (25)


Enhancing PPPPPYYY (26)


Enhancing PPPPPYYY (27)


Enhancing PPPPPYYY (28)


Enhancing PPPPPYYY (29)


Enhancing PPPPPYYY (3)


Enhancing PPPPPYYY (30)


Enhancing PPPPPYYY (4)


Enhancing PPPPPYYY (5)


Enhancing PPPPPYYY (6)


Enhancing PPPPPYYY (7)


Enhancing PPPPPYYY (8)


Enhancing PPPPPYYY (9)


Enhancing PYPYPPYY (1)


Enhancing PYPYPPYY (10)


Enhancing PYPYPPYY (11)


Enhancing PYPYPPYY (12)


Enhancing PYPYPPYY (13)


Enhancing PYPYPPYY (14)


Enhancing PYPYPPYY (15)


Enhancing PYPYPPYY (16)


Enhancing PYPYPPYY (1

# Course Segmentation

In [9]:
# Check doc strings for more information
seg_net = TracerUniversalB7(device='cuda',
              batch_size=16)

fba = FBAMatting(device='cpu',
                 input_tensor_size=2048,
                 batch_size=16)

trimap = TrimapGenerator()

preprocessing = PreprocessingStub()

postprocessing = MattingMethod(matting_module=fba,
                               trimap_generator=trimap,
                               device='cpu')

interface = Interface(pre_pipe=None,
                      post_pipe=None,
                      seg_pipe=seg_net)


img_set = [0]*8*len(img_nameList)

print(f'\nLoading Images\n')
for img_num, img_name in enumerate(img_nameList):
    for tube_num in range(8):
      img_set[8*img_num + tube_num] = PIL.Image.open(f"{path}\Split_enhanced\{img_name}_Tube{tube_num+1}_enhanced.jpg")

print(f'\nStart Course Segmentation\n')
img_set_seg = interface(img_set)
print(f'\nCourse Segmentation Finished\n')

for img_num, img_name in enumerate(img_nameList):
    for tube_num in range(8):
      img_set_seg[8*img_num + tube_num].save(f"{path}\courseSeg_tube\{img_name}_Tube{tube_num+1}_courseg.png")

print(f'\nImages Saved\n')



# for img_num, img_name in enumerate(img_nameList):
  
#   fig, ax = plt.subplots(2, 8, figsize=(10, 2), dpi=300)
#   for tube_num in range(8):

#     ax[0,tube_num].imshow(img_set[8*img_num + tube_num])
#     ax[1,tube_num].imshow(img_set_seg[8*img_num + tube_num])
#     ax[0,tube_num].axis('off')
#     ax[1,tube_num].axis('off')

#   plt.tight_layout()
#   fig.savefig(f"{path}\courseSeg_pic\{img_name}_courseg.png")
#   plt.close()



Loading Images


Start Course Segmentation


Course Segmentation Finished


Images Saved



# Fine Segmetation

In [10]:

for img_name in img_nameList:

  print(f'\nSegmenting {img_name}\n')

  # fig, ax = plt.subplots(3, 8, figsize=(10, 4), dpi=300)
  for num in range(8):
    img_split = cv.imread(f"{path}/Split/{img_name}_Tube{num+1}.jpg")
    img_courseg_enhanced = cv.imread(f"{path}/courseSeg_tube/{img_name}_Tube{num+1}_courseg.png", -1)
    img_courseg_bgra = cv.cvtColor(img_split, cv.COLOR_BGR2BGRA)
    idx = img_courseg_enhanced[:,:,3]==0
    img_courseg_bgra[idx,:] = [130,130,130,0]

    img_courseg_bgr = cv.cvtColor(img_courseg_bgra, cv.COLOR_BGRA2BGR)
    img_courseg_hsv = cv.cvtColor(img_courseg_bgr, cv.COLOR_RGB2HSV)

    ret, mask_fineseg = cv.threshold(img_courseg_hsv[:,:,1],0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

    kernal = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
    mask_fineseg = cv.morphologyEx(mask_fineseg, cv.MORPH_OPEN, kernal, iterations=2)

    img_fineseg = cv.cvtColor(img_split, cv.COLOR_BGR2BGRA)
    img_fineseg[:,:,3] = mask_fineseg

    # mask_joint = (img_courseg_bgra[:,:,3].astype('int') + mask_fineseg.astype('int')) == 510

    # kernal = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
    # mask_joint = cv.morphologyEx(mask_joint.astype('u8'), cv.MORPH_OPEN, kernal, iterations=5)



    # img_fineseg = img_courseg_bgra.copy()
    # img_fineseg[:,:,3] = np.zeros(img_fineseg.shape[:2])
    # img_fineseg[mask_joint,3] = 255

    cv.imwrite(f"{path}/fineSeg_tube/{img_name}_Tube{num+1}_fineseg.png", img_fineseg)

  #   ax[0,num].imshow(cv.cvtColor(img_split, cv.COLOR_BGR2RGB))
  #   ax[1,num].imshow(cv.cvtColor(img_courseg_bgra, cv.COLOR_BGRA2RGBA))
  #   ax[2,num].imshow(cv.cvtColor(img_fineseg, cv.COLOR_BGR2RGBA))
  #   ax[0,num].axis('off')
  #   ax[1,num].axis('off')
  #   ax[2,num].axis('off')

  # plt.tight_layout()

  # fig.savefig(f"{path}/fineSeg_pic/{img_name}_fineseg.png")

  # plt.close()


Segmenting PPPPPYYY (1)


Segmenting PPPPPYYY (10)


Segmenting PPPPPYYY (11)


Segmenting PPPPPYYY (12)


Segmenting PPPPPYYY (13)


Segmenting PPPPPYYY (14)


Segmenting PPPPPYYY (15)


Segmenting PPPPPYYY (16)


Segmenting PPPPPYYY (17)


Segmenting PPPPPYYY (18)


Segmenting PPPPPYYY (19)


Segmenting PPPPPYYY (2)


Segmenting PPPPPYYY (20)


Segmenting PPPPPYYY (21)


Segmenting PPPPPYYY (22)


Segmenting PPPPPYYY (23)


Segmenting PPPPPYYY (24)


Segmenting PPPPPYYY (25)


Segmenting PPPPPYYY (26)


Segmenting PPPPPYYY (27)


Segmenting PPPPPYYY (28)


Segmenting PPPPPYYY (29)


Segmenting PPPPPYYY (3)


Segmenting PPPPPYYY (30)


Segmenting PPPPPYYY (4)


Segmenting PPPPPYYY (5)


Segmenting PPPPPYYY (6)


Segmenting PPPPPYYY (7)


Segmenting PPPPPYYY (8)


Segmenting PPPPPYYY (9)


Segmenting PYPYPPYY (1)


Segmenting PYPYPPYY (10)


Segmenting PYPYPPYY (11)


Segmenting PYPYPPYY (12)


Segmenting PYPYPPYY (13)


Segmenting PYPYPPYY (14)


Segmenting PYPYPPYY (15)


Segmenting